Analyzing large volumes of text by clustering documents into topics, unlabled data

Its very difficult to analyze unsupervised learning

##  Latent dirichlet allocation
Uses prob distribution
Assumptions

1. Document with similar topics uses similar groups of words
2. Latent topics can then be found by searching for groups of words that freq
occur together in documents across the corpus

Mathematical assumption

1. Documents are probability distribution over latent topics
2. Topics themselves are probability disributions of words

It depends on the user to name the topics.
It assumes that the documents are produced in the following fashion:

 1. Choose a topic mixture for the document.
 example 60% business 20% politics 10% food.
 2. Generate each word in the document by:
 first picking a topic according to the multinomial distribution that you previously sampled(60,20,10)
 3. Assuming this generative model for a collection of documents,LDA then tries to backtrack from the documents to find a set of topics that are likely to have generated the collection

Now imagine we have a set of documents
We have to choose a fixed number of k topics to discover, and want to use
LDA to learn topic representation of each document and the words associated to
each topic

Go through each document, and randomly assign each word in the document to one of the k topics
This random assignment already gives you both topic representation of all the document and word distributions of all the topics (nite these initial random topic wont make sense)

Now we iterate over every word in every document to improves these topics
For every word in every document and for each topic t we calculate:
    p(t|d)= proportion of words in documents d that are assigned to topic t

For every word in every document and for each topic t we calculate:
    p(w|t)= the proportion of assignment to topic t over all documents that come from this word w

Reassign W a new topic where we choose topic t with probability p(t|d)*p(w|t)

This is essentially the probability that topic t generated word w

After repeating several times we get a resonable output
At the end we have each document assigned to a topic
We also can search for the words that have the highest probability of being assigned to a topic

We get:
    1. document assigned to topic 4
    2. Most common words for topic 4
    3. Its up to the user to interpret the topics

In [7]:
import pandas as pd
npr = pd.read_csv('npr.csv')

In [8]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

In [10]:
len(npr)

11992

In [12]:
npr.shape

(11992, 1)

# Preprocessing

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

Convert a collection of text documents to a matrix of token counts. #BOW


**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [16]:
vec = CountVectorizer(max_df=0.9,min_df=2, stop_words = 'english')

In [17]:
doc_term_matr=vec.fit_transform(npr['Article'])

In [19]:
doc_term_matr

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

In [21]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [22]:
LDA.fit(doc_term_matr)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=7, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

# Grab the voc of words


In [23]:
len(vec.get_feature_names())

54777

In [25]:
vec.get_feature_names()[50000]

'transcribe'

In [27]:
import random
random_word = random.randint(0,54777)
vec.get_feature_names()[random_word]

'surrogacy'

# Grab the topics

In [29]:
len(LDA.components_)

7

In [30]:
LDA.components_.shape

(7, 54777)

 # Grab the highest probability words per topic

In [32]:
single_topic = LDA.components_[0]

In [33]:
import numpy as np
arr = np.array([10,200,1])
arr.argsort()

array([2, 0, 1], dtype=int64)

Argsort returns index position sorted from least to greatest top 10 values  that is last 10 values of argsort

In [35]:
single_topic.argsort()[-10:]#hisghest prob words

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993], dtype=int64)

In [36]:
top_ten_words = single_topic.argsort()[-10:]

In [39]:
for index in top_ten_words:
    print(vec.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


# top 15 term for each 7 topics

In [41]:
for index,topic in enumerate(LDA.components_):
    print(f"Top 15 words for TOPIC #{index}")
    print([vec.get_feature_names()[index] for index in topic.argsort()[-15:]])
    print("\n")
    print("\n")

Top 15 words for TOPIC #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']




Top 15 words for TOPIC #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']




Top 15 words for TOPIC #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']




Top 15 words for TOPIC #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']




Top 15 words for TOPIC #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']




Top 15 words for TOPIC #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'know', 'think', 'people', 'ju

In [42]:
topic_results = LDA.transform(doc_term_matr)

In [43]:
topic_results[0].argmax()

1

In [44]:
npr['topic']= topic_results.argmax(axis = 1)